In [ ]:
import os
import json
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device)

In [ ]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = image_file
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [ ]:
path = "OpenGVLab/InternVL2-1B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

In [ ]:
#check where the tensors are allocated
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
def generate_response(question, image):
    """ Prompt model with question regarding image and generate response.
    Args:
        question (str): question regarding the image content
        image_path (str): PIL image object
    
    Returns:
        response (str): model's response to the question
    """
    if image:
        pixel_values = load_image(image, max_num=12).to(torch.bfloat16).cuda() 
    generation_config = dict(max_new_tokens=1024, do_sample=True, max_time=90)
    response, _ = model.chat(tokenizer, None, question, generation_config, history=None, return_history=True)
    
    return response

# Choose which benchmark for evaluation

In [ ]:
# set benchmark. Options are POPE or HallusionBench 
benchmark = "POPE"

# set if we want the LVLM to generate a prompt for itself
double_prompting = False

### Load dataset


In [ ]:
if benchmark == "POPE":
    # Load POPE benchmark dataset
    dataset = load_dataset("lmms-lab/POPE", "default")
    dataset = dataset['test'].filter(lambda x: x['category'] == 'adversarial')
    tags_path =  "../results/pope_tags.json"

elif benchmark == "HallusionBench":
    dataset = load_dataset("lmms-lab/HallusionBench", "default")
    dataset = dataset['image']
    tags_path = "../results/hallusionBench_tags.json"

dataset

In [ ]:
def tags_to_dict(filepath):
    """
    Access RAM tags from json file and convert to dictionary.
    """
    filepath = filepath
    ram_data = {}
    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip().rstrip(",")
    
            data_entry = json.loads(line)
    
            ram_data.update(data_entry)
    return ram_data

ram_data = tags_to_dict(tags_path)

In [ ]:
def obtain_attributes(img_src):
    """
    Returns the attributes identified by RAM.
    """
    injection = "This image has these attributes: "
    image_attrs = ram_data[img_src]
    image_attrs = image_attrs.replace('|',' ').split()
    injection = "This image has these attributes: "
    for i in range(len(image_attrs)):
        if i == len(image_attrs) - 2:
            injection = injection + image_attrs[i]+", and "
        elif i == len(image_attrs) - 1:
            injection = injection + image_attrs[i] +". "
        else:
            injection = injection + image_attrs[i] + ", "
    return injection

In [ ]:
def inject_info(img_src, question):
    """
    Injects prompt with any needed information. So given question, it will tell the lvlm also what it contains.
    Should fine-tune prompt later.
    """
    image_attrs = ram_data[img_src]
    image_attrs = image_attrs.replace('|',' ').split()
    injection = obtain_attributes(img_src)
    injection = f"Objects in image: {injection}. {question}"
    
    return injection

In [ ]:
def llm_prompting(img_src, image, question):
    """
    Prompt the model to describe the image using the tags generated by RAM.
    Returns an augmented prompt containing the LVLM's own description of the image.
    """
    attrs = obtain_attributes(img_src)
    attrs = attrs[0].lower() + attrs[1:]
    
    prompt = f"The following tags can be used to describe the image: {attrs}. Use the tags to help you describe the image. What do you see?"
    
    llm_prompt = generate_response(prompt, image)
    prompt = f"{llm_prompt} Answer the question using a single word or phrase. {question}"
    return prompt

## Generate LVLM responses

In [ ]:
print(f"Generating responses for: {benchmark}. Double prompting: {double_prompting}")

In [ ]:
double_prompting = True

In [ ]:
if benchmark == "POPE":

    for idx in range(len(dataset)):
        question = dataset['question'][idx]
        image = dataset['image'][idx]
        img_source = dataset['image_source'][idx]
        
        if double_prompting:
            internvl_pope_output_path = os.path.join(os.curdir, "../results/internvl_2xPrompting_RAM_pope_responses.json")
            prompt = llm_prompting(img_source, image, question)
        else:
            internvl_pope_output_path = os.path.join(os.curdir, "../results/internvl_RAM_pope_responses.json")
            prompt = inject_info(img_source, question)
        response = generate_response(prompt, image)
        
        with open(internvl_pope_output_path, 'a') as f:
            json.dump({"question": question, "answer": response}, f)
            f.write('\n')

    print(f"InternVL's responses have been saved to {internvl_pope_output_path}")

elif benchmark == "HallusionBench":
    for idx in range(len(dataset)):
        category = dataset['category'][idx]
        subcategory = dataset['subcategory'][idx]
        visual_input = dataset['visual_input'][idx]
        set_id = dataset['set_id'][idx]
        figure_id = dataset['figure_id'][idx]
        sample_note = dataset['sample_note'][idx]
        question_id = dataset['question_id'][idx]
        question = dataset['question'][idx]
        gt_answer_details = dataset['gt_answer_details'][idx]
        filename = dataset['filename'][idx]
        image = dataset['image'][idx]

        if double_prompting:
            internvl_RAM_hallusion_output_path =  os.path.join(os.curdir, "../results/internvl_2xPrompting_RAM_hallusion_responses.json")
            prompt = llm_prompting(filename, image, question)
        else:
            internvl_RAM_hallusion_output_path =  os.path.join(os.curdir, "../results/internvl_RAM_hallusion_responses.json")
            prompt = inject_info(filename, question)
            
        model_prediction = generate_response(prompt, image)
        
        with open(internvl_RAM_hallusion_output_path, 'a') as f:
            json.dump({
                "category": category,
                "subcategory": subcategory,
                "visual_input": visual_input,
                "set_id": set_id,
                "figure_id": figure_id,
                "sample_note": sample_note,
                "question_id": question_id,
                "question": question,
                "gt_answer_details": gt_answer_details,
                "model_prediction": model_prediction
            }, f)
            f.write('\n')
        print(f"Question {question_id} has been processed.")

    print(f"InternVL's responses have been saved to {internvl_RAM_hallusion_output_path}")